In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [52]:
import gensim
import pandas
import pandas
import numpy
import os
from scipy.linalg import norm
from gensim.models import KeyedVectors
from utilities import *

In [53]:
spine_glove = KeyedVectors.load_word2vec_format('../gridserach_Glove/result_vector/pmc_glove_spine_h1000_rl_2' , binary=False)

In [8]:
data = pd.read_csv('./category_med.csv').drop('Unnamed: 0', axis=1).drop_duplicates()

In [9]:
data.head()

,STR,STY
0,mptp,Organic Chemical
1,mptp,Hazardous or Poisonous Substance
2,mda,Organic Chemical
3,mda,Hazardous or Poisonous Substance
4,abbreviations,Intellectual Product


In [46]:
data_1 ={i[0]:list(i[1]) for i in list(data.groupby('STY').STR) if (len(i[1]) > 5) and (len(i[1]) < 250)}

In [47]:
cat = pd.DataFrame([list(data_1.keys()), data_1.values()]).T

In [48]:
max([len(i) for i in data_1.values()])

209

In [49]:
cat

,0,1
0,Acquired Abnormality,"[stump, corn, corning, nodule, nodules, catara..."
1,Activity,"[burial, consumption, doping, immigration, int..."
2,Age Group,"[adult, adults, child, children, infant, infan..."
3,Amphibian,"[amphibians, amphibian, frogs, frog, xenopus, ..."
4,Anatomical Abnormality,"[contracture, diverticulum, fistula, fistulas,..."
...,...,...
88,Substance,"[air, dust, fossils, fossil, freshwater, glue,..."
89,Temporal Concept,"[adolescence, youth, appointments, appointment..."
90,Tissue,"[callus, cartilage, dermis, endocardial, endot..."
91,Virus,"[adenoviruses, adenovirus, adenoviral, bacteri..."


In [64]:
def score_cat_inter(emb, cat, _lambda = 1, dim = 1000):
    import ast
    cats = cat[1]
    scores = np.zeros((1000,len(cat)))
    from tqdm import tqdm

    for i in tqdm(range(0,dim)):
        temp = topk_dim(emb, dim= i, k= 250 * _lambda)
        for num, voc in enumerate(cats): 
            scores[i, num] = len(set(temp[:len(voc) * _lambda]) & set(voc)) / len(voc)
    return scores.max(axis =0).mean()




In [75]:
def topk_dim(word_embedding, dim, reverse = 0, k = 10):
    if reverse == 1:
        top_words = normalize(word_embedding)[...,dim].argsort()[:k]
    else :
        top_words = normalize(word_embedding)[...,dim].argsort()[-k:][::-1]
    temp = []
    for i in top_words:
        temp.append(list(word_embedding.vocab.keys())[i])
    return temp


In [81]:
def score_cat_inter(emb, cat, _lambda = 1, dim = 1000):
    import ast
    cats = cat[1]
    scores = np.zeros((1000,len(cat)))
    from tqdm import tqdm

    for i in tqdm(range(0,dim)):
        top = topk_dim(emb, dim= i, k= 250 * _lambda)
        buttum = topk_dim(emb, dim= i, reverse = 1, k= 250 * _lambda)
        for num, voc in enumerate(cats): 
            pos = len(set(top[:len(voc) * _lambda]) & set(voc)) / len(voc)
            neg = len(set(buttum[:len(voc) * _lambda]) & set(voc)) / len(voc)
            scores[i, num] = max([pos,neg])
    return scores.max(axis =0).mean()




In [82]:
score_cat_inter(spine_glove, cat)

100%|██████████| 1000/1000 [03:24<00:00,  4.89it/s]


0.18123898270447694

In [84]:
spow_glove = KeyedVectors.load_word2vec_format('../gridserach_Glove/spow_result_vectors/spow_glove_h1000_1.txt' , binary=False)

In [85]:
score_cat_inter(spow_glove, cat)

100%|██████████| 1000/1000 [03:24<00:00,  4.89it/s]


0.09951058449144706

In [70]:
glove = KeyedVectors.load_word2vec_format('../GloVe/GloVe_pmc_300_min20_win16_20k.txt' , binary=False)

In [86]:
score_cat_inter(glove, cat, dim= 300)

100%|██████████| 300/300 [00:26<00:00, 11.20it/s]


0.08683428681392534

In [88]:
spine_sg= KeyedVectors.load_word2vec_format('../gridserach_SG/result_vector/pmc_sg_spine_h1000_psl_0.2' , binary=False)

In [90]:
score_cat_inter(spine_sg, cat)

100%|██████████| 1000/1000 [03:22<00:00,  4.94it/s]


0.16776261472511778

In [93]:
9spow_sg= KeyedVectors.load_word2vec_format('../gridserach_SG/spow_result_vectors/spow_sg_h1000_1.txt' , binary=False)

In [95]:
score_cat_inter(spow_sg, cat)

100%|██████████| 1000/1000 [03:25<00:00,  4.85it/s]


0.12366172221548145

In [96]:
sg= KeyedVectors.load_word2vec_format('../Skip_Gram/sg_pmc_300_min20_win16_20k_2.txt' , binary=False)

In [97]:
score_cat_inter(sg, cat, dim= 300)

100%|██████████| 300/300 [00:27<00:00, 10.96it/s]


0.09694318598191941

In [102]:
np.array([len(i) for i in cat[1]]).mean()

62.053763440860216